In [1]:
import pandas as pd
import numpy as np

In [2]:
pretrain_mut = pd.read_csv('../data/pdx/pdx_mut.csv', index_col=0)
pretrain_mut.head()

,A2LD1,ABCA4,ABCC13,ABHD13,ABTB1,ABTB2,ACBD7,ACTR6,ADAM17,ADAM3A,...,FAM196A,GPR123,MIR202,NKX6-2,PCDHGC5,SEPT2,SEPT6,SEPT8,UTF1,VENTX
X-1004,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
X-1008,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1298,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
X-1349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X-1371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = pd.read_csv('../data/pdx/PDX_klaegerscan_data_for_ml_CRC.csv')
data.head()

,Model,Treatment,Treatment.target,Treatment.type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,...,mut_ZDHHC11_0,mut_ZDHHC11_1,mut_ZNF219_0,mut_ZNF219_1,mut_ZNF384_0,mut_ZNF384_1,mut_LOC728888_1,mut_LOC728888_0,mut_NBPF12_0,mut_NBPF12_1
0,X1027,BKM120,"PIK3CA,PIK3CB,PIK3CG,PIK3CD,panPI3K",single,157.2,21,66.0,14,11.6,24,...,0,1,0,1,1,0,0,1,1,0
1,X1027,BYL719,PIK3CA,single,173.2,21,15.0,11,15.5,21,...,0,1,0,1,1,0,0,1,1,0
2,X1027,BYL719,PIK3CA,single,173.2,21,15.0,11,15.5,21,...,0,1,0,1,1,0,0,1,1,0
3,X1027,BYL719 + encorafenib,"PIK3CA,BRAF",combo,374.5,17,174.4,13,7.0,17,...,0,1,0,1,1,0,0,1,1,0
4,X1027,BYL719 + binimetinib,"PIK3CA,MAPK",combo,168.6,13,104.1,13,7.0,13,...,0,1,0,1,1,0,0,1,1,0


In [4]:
#get columns that do not start with 'mut_' or 'act_' or 'cna_' or 'exp_'
mut_col = [col for col in data.columns if 'mut_' in col]
act_col = [col for col in data.columns if 'act_' in col]
cna_col = [col for col in data.columns if 'cnv_' in col]
exp_col = [col for col in data.columns if 'exp_' in col]
id_cols = []
for col in data.columns:
    if col not in mut_col and col not in act_col and col not in cna_col and col not in exp_col:
        id_cols.append(col)
len(id_cols)


17

In [5]:
data_id = data[id_cols]
data_id.head()

,Model,Treatment,Treatment.target,Treatment.type,BestResponse,Day_BestResponse,BestAvgResponse,Day_BestAvgResponse,TimeToDouble,Day_Last,ResponseCategory,NonPD,CRorPR,TumorType,RespScaled,logSurv,logSurvScaled
0,X1027,BKM120,"PIK3CA,PIK3CB,PIK3CG,PIK3CD,panPI3K",single,157.2,21,66.0,14,11.6,24,PD,0,0,CRC,1.186739,2.451005,2.534604
1,X1027,BYL719,PIK3CA,single,173.2,21,15.0,11,15.5,21,PD,0,0,CRC,0.269713,2.740840,2.834324
2,X1027,BYL719,PIK3CA,single,173.2,21,15.0,11,15.5,21,PD,0,0,CRC,0.269713,2.740840,2.834324
3,X1027,BYL719 + encorafenib,"PIK3CA,BRAF",combo,374.5,17,174.4,13,7.0,17,PD,0,0,CRC,3.135867,1.945910,2.012281
4,X1027,BYL719 + binimetinib,"PIK3CA,MAPK",combo,168.6,13,104.1,13,7.0,13,PD,0,0,CRC,1.871811,1.945910,2.012281


In [6]:
data_act = data[act_col]
data_act['CRorPR'] = data['CRorPR']
data_act['CRorPR'].value_counts()


/tmp/ipykernel_2903815/2884522922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['CRorPR'] = data['CRorPR']


CRorPR
0    306
1     23
Name: count, dtype: int64

In [7]:
data_act.shape

(329, 818)

In [8]:
clin = pd.read_csv('../data/crc_ib_mut_cna_act_clin.csv', index_col=0)
clin.head()

,id_institution,id_drugs_list,OS,PFS,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,...,act_PTK2B,act_RPS6KB1,act_STK10,act_STK11,act_STK24,act_STK3,act_STK38,act_STK38L,act_STK4,act_YES1
GENIE-DFCI-000971,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-001038,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002499,DFCI,Cabozantinib Smalate,1,0.0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002507,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
GENIE-DFCI-002561,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
mut_clin = [col for col in clin.columns if 'mut_' in col]
mut_clin_genes = [col.split('_')[1] for col in mut_clin]
mut_data = [col for col in pretrain_mut.columns]
overlap = [col for col in mut_clin_genes if col in mut_data]
len(overlap)

219

In [10]:
pretrain_mut = pretrain_mut[overlap]
mut_clin = ['mut_' + col for col in overlap]
clin_mut = clin[mut_clin]
print(pretrain_mut.shape, clin_mut.shape)

(399, 219) (139, 219)


In [11]:
for col in mut_clin:
    data_act[col] = np.NaN

/tmp/ipykernel_2903815/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act[col] = np.NaN
/tmp/ipykernel_2903815/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act[col] = np.NaN
/tmp/ipykernel_2903815/1900429651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [12]:
pretrain_mut.index = pretrain_mut.index.str.replace('-', '')
pretrain_mut.head()

,CDK4,CCND3,CDH1,CDK8,GNAS,PRKAR1A,MLH1,PNRC1,ETV1,TSC1,...,CDKN1A,ROS1,DAXX,TSC2,RAD51C,ARID1A,AURKB,U2AF1,REL,BRCA1
X1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1298,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
X1349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data_act.head()

,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_CDKN1A,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1
0,1.00,1,0.68,1.0,1.00,1.0,0.82,0.75,0.72,0.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.00,1,1.00,1.0,1.00,1.0,1.00,0.93,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data_act['line'] = data['Model']
data_act.head()

/tmp/ipykernel_2903815/3177637675.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_act['line'] = data['Model']
/tmp/ipykernel_2903815/3177637675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act['line'] = data['Model']


,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1,line
0,1.00,1,0.68,1.0,1.00,1.0,0.82,0.75,0.72,0.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1027
1,1.00,1,1.00,1.0,1.00,1.0,1.00,0.93,1.00,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1027
2,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1027
3,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1027
4,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X1027


In [15]:
#fill in the 'mut_' columns of data_act with the pretrain_mut, matching the index of pretrain_mut with 'line' in data_act
for i in range(data_act.shape[0]):
    line = data_act.iloc[i]['line']
    if line in pretrain_mut.index:
        data_act.loc[i, mut_clin] = pretrain_mut.loc[line].values

In [16]:
data_act.head()

,act_AAK1,act_ABL1,act_ABL1_nonphosphorylated,act_ABL1_phosphorylated,act_ABL2,act_ACVR1,act_ACVR1B,act_ACVR2A,act_ACVR2B,act_ACVRL1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1,line
0,1.00,1,0.68,1.0,1.00,1.0,0.82,0.75,0.72,0.99,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,X1027
1,1.00,1,1.00,1.0,1.00,1.0,1.00,0.93,1.00,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,X1027
2,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,X1027
3,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,X1027
4,0.75,1,0.53,0.1,0.96,1.0,0.93,1.00,0.93,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,X1027


In [17]:
data_act.shape

(329, 1038)

In [18]:
data_act.set_index('line', inplace=True)
data_act.to_csv('../data/pdx/pdx_act_mut_cpr.csv')

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [52]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
aurocs = cross_val_score(rf, data_act.drop(['line', 'CRorPR'], axis=1), data_act['CRorPR'], scoring='roc_auc', cv=5)
aurocs

array([0.71370968, 0.66393443, 0.54590164, 0.48852459, 0.90983607])

In [53]:
np.mean(aurocs)

0.664381279746166

In [20]:
act_cols = [col for col in data_act.columns if 'act_' in col]
#combine the 'act_' columns with the 'mut_' columns
act_mut_cols = act_cols + mut_clin
len(act_mut_cols)

1036

In [21]:
train = data_act.copy()
test = clin[act_mut_cols]
test['OS'] = clin['OS']
test['PFS'] = clin['PFS']
test.to_csv('../data/crc_ib_mut_act.csv')

/tmp/ipykernel_2903815/1423953895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['OS'] = clin['OS']
/tmp/ipykernel_2903815/1423953895.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['PFS'] = clin['PFS']


In [68]:
X_train = train.drop(['line', 'CRorPR'], axis=1)
y_train = train['CRorPR']
X_test = test.drop(['OS'], axis=1)
y_test = test['OS']

In [69]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.584733893557423

In [79]:
train = data_act.copy()
test = clin[act_mut_cols]
test['PFS'] = clin['PFS']
test = test.dropna(subset=['PFS'])
print(train.shape, test.shape)

(329, 1038) (133, 1038)


/tmp/ipykernel_2901307/3769482909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['PFS'] = clin['PFS']


In [80]:
X_train = train.drop(['line', 'CRorPR'], axis=1)
y_train = train['CRorPR']
X_test = test.drop(['PFS', 'OS'], axis=1)
y_test = test['PFS']
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.4570990806945863

In [81]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
aurocs = cross_val_score(rf, X_test, y_test, scoring='roc_auc', cv=5)
aurocs

array([0.45061728, 0.57407407, 0.62962963, 0.62152778, 0.60130719])

In [82]:
np.mean(aurocs)

0.5754311909949166